# Importing Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchvision.transforms import v2
from torch.utils.data import DataLoader, TensorDataset
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import timm
import random
import shutil

In [2]:
def import_dataset(clip_limit=None,tilegridsize=None,Clahe = False):
    
    if Clahe:
        clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tilegridsize)
    
    train_dict = {}
    val_dict = {}
    test_dict = {}
    
    train_path = "/kaggle/input/knee-osteoarthritis-dataset-with-severity/train/"
    val_path = "/kaggle/input/knee-osteoarthritis-dataset-with-severity/val/"
    test_path = "/kaggle/input/knee-osteoarthritis-dataset-with-severity/test/"
    
    for label in os.listdir(train_path):
        train_dict[int(label)] = os.listdir(train_path + label)
    for label in os.listdir(test_path):
        test_dict[int(label)] = os.listdir(test_path + label)
    for label in os.listdir(val_path):
        val_dict[int(label)] = os.listdir(val_path + label)
        
    train_images = {}
    val_images = {}
    test_images = {}
    
    for keys in train_dict.keys():  
        normal_train_images = [np.array(Image.open(train_path + str(keys) + "/" + images)) for images in train_dict[keys]]
        normal_test_images = [np.array(Image.open(test_path + str(keys) + "/" + images)) for images in test_dict[keys]]
        normal_val_images = [np.array(Image.open(val_path + str(keys) + "/" + images)) for images in val_dict[keys]]
        
        if Clahe:
            clahe_train_images = [clahe.apply(np.array(Image.open(train_path + str(keys) + "/" + images))) for images in train_dict[keys]]
            clahe_test_images = [clahe.apply(np.array(Image.open(test_path + str(keys) + "/" + images))) for images in test_dict[keys]]
            clahe_val_images = [clahe.apply(np.array(Image.open(val_path + str(keys) + "/" + images))) for images in val_dict[keys]]
            
            train_images[keys] =  clahe_train_images
            test_images[keys] =  clahe_test_images
            val_images[keys] =  clahe_val_images
            
        else:
            train_images[keys] = normal_train_images
            test_images[keys] = normal_test_images
            val_images[keys] = normal_val_images 
            
            
    return train_images,test_images,val_images

In [3]:
def create_tensors(train_images,test_images,val_images):
    
    trlabel0 = [0 for i in range(len(train_images[0]))]
    trlabel1 = [1 for i in range(len(train_images[1]))]
    trlabel2 = [2 for i in range(len(train_images[2]))]
    trlabel3 = [3 for i in range(len(train_images[3]))]
    trlabel4 = [4 for i in range(len(train_images[4]))]
    
    tslabel0 = [0 for i in range(len(test_images[0]))]
    tslabel1 = [1 for i in range(len(test_images[1]))]
    tslabel2 = [2 for i in range(len(test_images[2]))]
    tslabel3 = [3 for i in range(len(test_images[3]))]
    tslabel4 = [4 for i in range(len(test_images[4]))]
    
    vlabel0 = [0 for i in range(len(val_images[0]))]
    vlabel1 = [1 for i in range(len(val_images[1]))]
    vlabel2 = [2 for i in range(len(val_images[2]))]
    vlabel3 = [3 for i in range(len(val_images[3]))]
    vlabel4 = [4 for i in range(len(val_images[4]))]
    
    
    training_image  = torch.tensor(torch.cat((torch.tensor(train_images[0]),torch.tensor(train_images[1]),torch.tensor(train_images[2]),torch.tensor(train_images[3]),torch.tensor(train_images[4])),0),dtype = torch.float32)
    training_labels = torch.tensor(torch.cat((torch.tensor(trlabel0),torch.tensor(trlabel1),torch.tensor(trlabel2),torch.tensor(trlabel3),torch.tensor(trlabel4)),0))
    training_image = training_image.view(training_image.shape[0],1,224,224)
    
    testing_image  = torch.tensor(torch.cat((torch.tensor(test_images[0]),torch.tensor(test_images[1]),torch.tensor(test_images[2]),torch.tensor(test_images[3]),torch.tensor(test_images[4])),0),dtype = torch.float32)
    testing_labels = torch.tensor(torch.cat((torch.tensor(tslabel0),torch.tensor(tslabel1),torch.tensor(tslabel2),torch.tensor(tslabel3),torch.tensor(tslabel4)),0))
    testing_image = testing_image.view(testing_image.shape[0],1,224,224)
    
    val_image  = torch.tensor(torch.cat((torch.tensor(val_images[0]),torch.tensor(val_images[1]),torch.tensor(val_images[2]),torch.tensor(val_images[3]),torch.tensor(val_images[4])),0),dtype = torch.float32)
    val_labels = torch.tensor(torch.cat((torch.tensor(vlabel0),torch.tensor(vlabel1),torch.tensor(vlabel2),torch.tensor(vlabel3),torch.tensor(vlabel4)),0))
    val_image = val_image.view(val_image.shape[0],1,224,224)
    
    return training_image,training_labels,testing_image,testing_labels,val_image,val_labels

In [4]:
def create_dataset(training_image,training_labels,testing_image,testing_labels,val_image,val_labels,batch_size = 4):
    train_dataset = TensorDataset(training_image,training_labels)
    val_dataset = TensorDataset(val_image,val_labels)
    test_dataset = TensorDataset(testing_image,testing_labels)
    
    train_loader = DataLoader(train_dataset,shuffle= True , batch_size = batch_size)
    val_loader = DataLoader(val_dataset,shuffle = True , batch_size = batch_size)
    test_loader = DataLoader(test_dataset,shuffle = True, batch_size = batch_size)
    
    return train_loader,test_loader,val_loader

In [5]:
original_model= torchvision.models.vgg16_bn(pretrained = True)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:03<00:00, 144MB/s]  


In [6]:
original_model.features[0] = nn.Conv2d(1,64,kernel_size = (3,3),stride = 1,padding = 1)

In [64]:
class NewModel(nn.Module):
    def __init__(self,model):
        super(NewModel,self).__init__()
        features = list(model.features.children())
                
        self.features = nn.Sequential(
                        *features,
                        )
        
        self.avgpool = nn.AvgPool2d(2)
        
        self.classifier = nn.Sequential(
                            nn.Linear(512*3,512),
                            nn.BatchNorm1d(512),
                            nn.ReLU(inplace = True),
                            nn.Dropout(p = 0.3,inplace = False),
                            nn.Linear(512,5),
                            )
        
            
    def forward(self,x):
            x = self.features(x)
            x = self.avgpool(x)
            x = x.view(x.shape[0],-1)
            
            x = self.classifier(x)
            
            return x

In [8]:
def create_model(original_model,no_of_layers =22):
    model = NewModel(original_model)

    filtered_state_dict = {k: v for k, v in original_model.state_dict().items() if k in model.state_dict() and v.shape == model.state_dict()[k].shape}
    
    model.load_state_dict(filtered_state_dict, strict=False)
    
    for i, params in enumerate(model.parameters()):
        if i < no_of_layers:
            params.requires_grad = False
    
    return model

In [9]:
def params(nmmodel,learning_rate = 0.0001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(nmmodel.parameters(),lr = learning_rate)
    
    return nmmodel,criterion,optimizer

In [10]:
def training(train_loader,val_loader,epochs = 10):
    training_loss = 0.0
    training_accuracy = 0
    validation_accuracy = 0
    device = 'cuda'
    for epoch in range(epochs):
        n_samples_train = 0
        n_samples_val = 0
        for i, (images, labels) in enumerate(train_loader):

            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            loss = criterion(output, labels)
            n_samples_train += labels.size(0)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            _, predicted = torch.max(output.data, 1)
            training_accuracy += (predicted == labels).sum().item()

            if (i + 1) % 512 == 0:
                print(f'Training - Epoch: {epoch + 1}, Batch: {i + 1}, Loss: {loss.item():.4f}')

        print(f'Training Accuracy - Epoch: {epoch + 1}: {training_accuracy * 100 / n_samples_train:.2f}%')
        training_accuracy = 0

        with torch.no_grad():
            for i, (val_images, val_labels) in enumerate(val_loader):

                val_images = val_images.to(device)
                val_labels = val_labels.to(device)
                val_output = model(val_images)
                _, val_predicted = torch.max(val_output.data, 1)
                validation_accuracy += (val_predicted == val_labels).sum().item()
                n_samples_val += val_labels.size(0)

        print(f'Validation Accuracy - Epoch: {epoch + 1}: {validation_accuracy * 100 / n_samples_val:.2f}%')
        validation_accuracy = 0

In [11]:
def testing(test_loader):
    device = 'cuda'
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        test_loss = 0.0
        test_accuracy = 0
        for i,(images, labels) in enumerate(test_loader):

            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs.data,labels)
            _, predicted = torch.max(outputs.data, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()
            test_accuracy = n_correct
            test_loss = loss.item()
            if((i+1)%50 == 0):

                test_accuracy = 0
                test_loss = 0
        acc = 100.0 * n_correct / n_samples
        print(f'Accuracy of the network on the {n_samples} test images:{acc:.4f}%')
    

In [12]:
train_images,test_images,val_images = import_dataset()

In [13]:
training_image,training_labels,testing_image,testing_labels,val_image,val_labels = create_tensors(train_images,test_images,val_images)

In [52]:
tr_image = training_image[:,:,65:180,:]
ts_image = testing_image[:,:,65:180,:]
v_image = val_image[:,:,65:180,:]

In [53]:
train_loader,test_loader,val_loader = create_dataset(tr_image,training_labels,ts_image,testing_labels,v_image,val_labels,batch_size = 4)

In [65]:
model = create_model(original_model,no_of_layers = 22)

In [66]:
model.to('cuda')

NewModel(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128

In [67]:
model,criterion,optimizer = params(model)

In [68]:
training(train_loader,val_loader,epochs = 8)

Training - Epoch: 1, Batch: 512, Loss: 0.9294
Training - Epoch: 1, Batch: 1024, Loss: 1.3976
Training Accuracy - Epoch: 1: 46.04%
Validation Accuracy - Epoch: 1: 49.88%
Training - Epoch: 2, Batch: 512, Loss: 1.5249
Training - Epoch: 2, Batch: 1024, Loss: 0.9221
Training Accuracy - Epoch: 2: 54.22%
Validation Accuracy - Epoch: 2: 53.63%
Training - Epoch: 3, Batch: 512, Loss: 1.8045
Training - Epoch: 3, Batch: 1024, Loss: 1.9423
Training Accuracy - Epoch: 3: 58.74%
Validation Accuracy - Epoch: 3: 56.42%
Training - Epoch: 4, Batch: 512, Loss: 0.8598
Training - Epoch: 4, Batch: 1024, Loss: 0.8330
Training Accuracy - Epoch: 4: 60.99%
Validation Accuracy - Epoch: 4: 58.47%
Training - Epoch: 5, Batch: 512, Loss: 1.5801
Training - Epoch: 5, Batch: 1024, Loss: 1.3210
Training Accuracy - Epoch: 5: 63.78%
Validation Accuracy - Epoch: 5: 58.11%
Training - Epoch: 6, Batch: 512, Loss: 0.9594
Training - Epoch: 6, Batch: 1024, Loss: 1.0034
Training Accuracy - Epoch: 6: 65.40%
Validation Accuracy - Epo

In [70]:
for i, params in enumerate(model.parameters()):
        if i < 22:
            params.requires_grad = True

In [71]:
training(train_loader,val_loader,epochs = 8)

Training - Epoch: 1, Batch: 512, Loss: 0.4473
Training - Epoch: 1, Batch: 1024, Loss: 0.3708
Training Accuracy - Epoch: 1: 67.24%
Validation Accuracy - Epoch: 1: 59.93%
Training - Epoch: 2, Batch: 512, Loss: 0.2606
Training - Epoch: 2, Batch: 1024, Loss: 0.7515
Training Accuracy - Epoch: 2: 70.68%
Validation Accuracy - Epoch: 2: 59.20%
Training - Epoch: 3, Batch: 512, Loss: 0.6003
Training - Epoch: 3, Batch: 1024, Loss: 0.3749
Training Accuracy - Epoch: 3: 73.05%
Validation Accuracy - Epoch: 3: 60.77%
Training - Epoch: 4, Batch: 512, Loss: 0.9556
Training - Epoch: 4, Batch: 1024, Loss: 0.3401
Training Accuracy - Epoch: 4: 77.50%
Validation Accuracy - Epoch: 4: 59.08%
Training - Epoch: 5, Batch: 512, Loss: 0.8115
Training - Epoch: 5, Batch: 1024, Loss: 0.2498
Training Accuracy - Epoch: 5: 80.74%
Validation Accuracy - Epoch: 5: 56.17%
Training - Epoch: 6, Batch: 512, Loss: 0.0306
Training - Epoch: 6, Batch: 1024, Loss: 0.4931
Training Accuracy - Epoch: 6: 84.25%
Validation Accuracy - Epo

In [72]:
testing(test_loader)

Accuracy of the network on the 1656 test images:64.7947%
